# Cancer Patient  
- Multiple office visit notes are read
- Compiled in a string (no particular chronological order)
- chatGPT parses and list chronologically with
    - Clinical Findings
    - Medications
    - Lab Values

In [1]:
import os
import openai
import webbrowser
import datetime 
import re
from datetime import datetime
import subprocess


from chatGPT_utilities import setup_OpenAI_API
from chatGPT_utilities import json_to_dict
from chatGPT_utilities import get_completion
from chatGPT_utilities import get_completion_from_messages_FULL
from chatGPT_utilities import read_file
from chatGPT_utilities import token_usage
from chatGPT_utilities import print_token_usage

openai.api_key = setup_OpenAI_API()
print(f'Current Directory:\n{os.getcwd()}')
starting_directory = os.getcwd()


starting_directory /Users/michael/Coding
Directory:/Users/michael/Coding
Current Directory:
/Users/michael/Coding


In [63]:
def write_to_file(file_name,patient_note,response):
    file = open(file_name + '.txt' ,'w') 
    now = datetime.now()

    formatted_date = now.strftime("%a %b %d at %H:%M")
    print(f'{formatted_date}')
    file.write('Created on: ' + formatted_date + '\n'*2 )
    
    file.write(patient_note + '\n'*2 )
    file.write(response + '\n'*2 )
    print(f'{file_name} was created in {os.getcwd()}')
    file.close() 
    return


In [2]:
def read_patient_notes():
    Cancer_Patient_Notes_file = 'Cancer_Patient_Notes'
    os.chdir(Cancer_Patient_Notes_file)

    list_of_notes = [i for i in os.listdir() if i.endswith('txt') and '_20' in i]

    compilation_of_notes = ''
    print(f'Processing')
    for num, name in enumerate(list_of_notes, start=1): 
        print("{0:3}: {1}".format(num, name))
    print('Note : The files are not necessarily in chronological order\n\n\n')

    for file_name in list_of_notes:

    # for date in ['Aug_20']:
        print(f'\033[91m                                     File Name: \033[1m{file_name}\033[0m') 
        interim_note = read_file(file_name)
        print(interim_note)
        compilation_of_notes= compilation_of_notes + '  \n\n' + interim_note
    print('\n\n\n\n\033[91m'+ '\033[1m\t\t\t--------------------------COMPILATION of NOTES--------------------------\033[0m') 
    print(compilation_of_notes)
    os.chdir('..')

    if os.getcwd() != starting_directory:
        os.chdir(starting_directory)
    assert os.getcwd() == starting_directory , 'We should be in starting directory'
    return compilation_of_notes

# Read clinical visit notes  

- Notes will not necessarily be retrieved in chronological order

![alt text](patient_notes_1.png "Here is Title")


In [3]:
compilation_of_notes = read_patient_notes()

Processing
  1: Nov_20.txt
  2: Oct_20.txt
  3: Sept_20.txt
  4: Aug_20.txt
Note : The files are not necessarily in chronological order



                                     File Name: Nov_20.txt
Date: November 20


Following the completion of induction chemotherapy, he is experiencing a mix of physical and emotional symptoms. Today his ANC 900, HCT 30

Physically, he is still recovering from the side effects of chemotherapy. Reports fatigue, weakness, and a general sense of low energy. Nausea and vomiting gradually subsiding. Has reduced appetite, and has lost 2 kg since initial diagnosis. Emotionally, the patient feeling some uncertainty as awaiting the results of the treatment response evaluation and bone marrow assessment. Will continue to provide supportive care, address any ongoing symptoms, and offer emotional support prn.


                                     File Name: Oct_20.txt
Date: Oct 20
This 65-year-old white male was 
diagnosed with acute myeloid leukemia (AML) in Au

# Extract Diagnosis and Synopsis of Visits   

From the compilation of notes below, extract the information relevant to patient's condition. 

Patient's Diagnosis:  
Date of Diagnosis:


List the dates of visits in chronological order and include:  
- Clinical findings
- lab values   
- Medications


For each visit:
- Date # 1 : 
    - Clinical Findings
    - Medications: 
    - HCT :
    - ANC : 


- Date # 2 : 
    - Clinical Findings
    - Medications: 
    - HCT :
    - ANC : 


- Date # 3 : 
    - Clinical Findings
    - Medications: 
    - HCT :
    - ANC : 


In [4]:
prompt_1 = f"""
Your task is to extract relevant information from 
clinical note. 

From the compilation of notes below, delimited by triple quotes \
extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and 
include clinical findings, lab values and any medications (including any chemotherapy medications) mentioned during that visit
 in the following format:

Date # 1 - 
    Clinical Findings    
    Medications:
    Lab Values:
    
Date # 2 - 
    Clinical Findings   
    Medications:…
Date # N - 
    Clinical Findings
    Medications:


Review: ```{compilation_of_notes}```
"""

print('\n\n\n\n\033[91m'+ '\033[1mSTART of QUERY\033[0m') 
start_time = datetime.datetime.now() 
print('start_time:',  start_time.strftime("%H:%M:%S"))
messages = [{"role": "user", "content": prompt_1}]
response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
    # print(response)

# response = get_completion(prompt)


end_time = datetime.datetime.now() 
print(response, '\n'*2)
print('end_time  :', end_time.strftime("%H:%M:%S"))
elapsed_time = end_time - start_time
print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
cost = print_token_usage(response_details)
print(f'Cost to run query: ${cost}')






START of QUERY
start_time: 13:38:55
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Date # 1 - August 20
    Clinical Findings: Fatigue and lethargy
    Medications: Statin and finasteride
    Lab Values: ANC 1500, HCT 27

Date # 2 - Sept 20
    Clinical Findings: Completed induction chemotherapy, achieved complete remission
    Medications: N/A
    Lab Values: N/A

Date # 3 - Oct 20
    Clinical Findings: Undergoing consolidation therapy, experiencing fatigue, lethargy, nausea, vomiting, decreased appetite, weight loss, myelosuppression
    Medications: Arabinosylcytosine and midostaurin
    Lab Values: ANC 1250, HCT 29

Date # 4 - Nov 20
    Clinical Findings: Experiencing physical and emotional symptoms, recovering from chemotherapy side effects, fatigue, weakness, low energy, nausea, vomiting, reduced appetite, weight loss
    Medications: N/A
    Lab Values: ANC 900, HCT 30 


end_time  : 13:39:06
Elapsed_time: 11.9 seconds
Cost to run query i

# Extract only the ANC & HCT


Patient's Diagnosis:  
Date of Diagnosis:


For each visit:
- Date # 1 : 
    - HCT :
    - ANC : 


- Date # 2 : 
    - HCT :
    - ANC : 


- Date # 3 : 
    - HCT :
    - ANC : 


In [60]:
print('\n\n\n\n\033[91m'+ '\033[1mSTART of QUERY\033[0m') 
start_time = datetime.datetime.now() 
print('start_time:',  start_time.strftime("%H:%M:%S"))
prompt_2 = f"""
Your task is to extract relevant information from clinical note. 

From the compilation of notes below, delimited by triple quotes \
extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and include the lab values in the following format
Date # 1 : 
   HCT :
   ANC : 
    
Date # 2 : 
   HCT :
   ANC : 

Date # N :
   HCT :
   ANC : 

Do not list any additional information in the synopsis other than lab values.
If no lab values are listed for a given date, indicate 'N/A'

Review: ```{compilation_of_notes}```
"""


messages = [{"role": "user", "content": prompt_2}]
response_details = get_completion_from_messages_FULL(messages)
response = response_details.choices[0].message["content"]
    # print(response)

# response = get_completion(prompt)
end_time = datetime.datetime.now() 
print(response, '\n'*2)
print('end_time  :', end_time.strftime("%H:%M:%S"))
elapsed_time = end_time - start_time
print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
cost = print_token_usage(response_details)
print(f'Cost to run query: ${cost}')





START of QUERY
start_time: 14:11:08
Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Dates of visits in chronological order with lab values:

Date: August 20
   HCT: 27
   ANC: 1500

Date: Sept 20
   HCT: N/A
   ANC: N/A

Date: Oct 20
   HCT: 29
   ANC: 1250

Date: November 20
   HCT: 30
   ANC: 900 


end_time  : 14:11:13
Elapsed_time: 5.412 seconds
Cost to run query is based on token usage, which is $0.002 / 1K tokens


Token Usage:


completion_tokens: 111
prompt_tokens    : 767
total_tokens     : 878
Cost to run query: $0.002


In [69]:
file_name = 'ANC_and_HCT'
write_to_file(file_name,compilation_of_notes,response)

_ = subprocess.call(['open',   file_name + '.txt'])

Mon Jun 26 at 14:14
ANC_and_HCT was created in /Users/michael/Coding


In [78]:
# print(compilation_of_notes)
# compilation_of_notes
_ = subprocess.call(['open',   file_name + '.txt'])
# .replace('x1b[1m\x1b[1m','')

# Assemble Prompts in a List of Prompts

In [12]:
prompt_list = [prompt_1,prompt_2]

In [13]:
for prompt in prompt_list:
    start_time =   datetime.datetime.now() 
    relevant_details = prompt.split("\n\nReview:")[0]
    print(f'\033[91m\033[1m {relevant_details}\033[0m') 
    messages = [{"role": "user", "content": prompt}]
    response_details = get_completion_from_messages_FULL(messages)
    response = response_details.choices[0].message["content"]
        # print(response)

    # response = get_completion(prompt)
    end_time = datetime.datetime.now() 
    
    end_time = datetime.datetime.now() 
    print(response, '\n'*2)
    print('end_time  :', end_time.strftime("%H:%M:%S"))
    elapsed_time = end_time - start_time
    print('Elapsed_time: ',elapsed_time.seconds,'.',int(elapsed_time.microseconds/1000),' seconds',sep='')
    cost = print_token_usage(response_details)
    print(f'Cost to run query: ${cost}\n\n') 
    

 
Your task is to extract relevant information from 
clinical note. 

From the compilation of notes below, delimited by triple quotes extract the information relevant to patient's condition. 

Patient's Diagnosis:
Date of Diagnosis:


List the dates of visits in chronological order and 
include clinical findings, lab values and any medications mentioned during that visit
 in the following format:

Date # 1 - 
    Clinical Findings    
    Medications:
    
Date # 2 - 
    Clinical Findings   
    Medications:…
Date # N - 
    Clinical Findings
    Medications:

Patient's Diagnosis: Acute Myeloid Leukemia (AML)
Date of Diagnosis: August 2020

Date: August 20
Clinical Findings: Fatigue, lethargy
Medications: Statin, finasteride
Lab Values: ANC 1500, HCT 27

Date: Sept 20
Clinical Findings: Completed induction chemotherapy, achieved complete remission
Medications: N/A
Lab Values: N/A

Date: Oct 20
Clinical Findings: Undergoing consolidation therapy, experiencing fatigue, lethargy, nausea,

# Bold Section

In [58]:
# print(compilation_of_notes)
print(compilation_of_notes)

  

Date: November 20


Following the completion of induction chemotherapy, he is experiencing a mix of physical and emotional symptoms. Today his ANC 900, HCT 30

Physically, he is still recovering from the side effects of chemotherapy. Reports fatigue, weakness, and a general sense of low energy. Nausea and vomiting gradually subsiding. Has reduced appetite, and has lost 2 kg since initial diagnosis. Emotionally, the patient feeling some uncertainty as awaiting the results of the treatment response evaluation and bone marrow assessment. Will continue to provide supportive care, address any ongoing symptoms, and offer emotional support prn.

  

Date: Oct 20
This 65-year-old white male was 
diagnosed with acute myeloid leukemia (AML) in August 2020. He underwent a round of induction chemotherapy in Aug 2020. Currently undergoing Consolidation therapy (in cycle 2 of 4). His ANC 1250, HCT 29
He is on Arabinosylcytosine and midostaurin 

Currently, the patient is experiencing common side

In [59]:
def find_substrings(text):
    pattern = r'Date[\s\S]*?\n+'
    matches = re.findall(pattern, text)
    return matches


substrings = find_substrings(compilation_of_notes)
if substrings:
    for substring in substrings:
        print(substring.strip())
else:
    print("Substrings not found.")

Date: November 20
Date: Oct 20
Date : Sept 20
Date : August 20


In [54]:
dates = [ i for i in substrings]
dates[0]

'Date: November 20\n\n\n'

In [56]:
for substr in substrings:
    compilation_of_notes = compilation_of_notes.replace(substr, '\033[1m' + substr + '\033[0m')

    
print(compilation_of_notes)

# print('\033[1m' + 'dfdf'+ '\033[0m')



  

Date: November 20


Following the completion of induction chemotherapy, he is experiencing a mix of physical and emotional symptoms. Today his ANC 900, HCT 30

Physically, he is still recovering from the side effects of chemotherapy. Reports fatigue, weakness, and a general sense of low energy. Nausea and vomiting gradually subsiding. Has reduced appetite, and has lost 2 kg since initial diagnosis. Emotionally, the patient feeling some uncertainty as awaiting the results of the treatment response evaluation and bone marrow assessment. Will continue to provide supportive care, address any ongoing symptoms, and offer emotional support prn.

  

Date: Oct 20
This 65-year-old white male was 
diagnosed with acute myeloid leukemia (AML) in August 2020. He underwent a round of induction chemotherapy in Aug 2020. Currently undergoing Consolidation therapy (in cycle 2 of 4). His ANC 1250, HCT 29
He is on Arabinosylcytosine and midostaurin 

Currently, the patient is experiencing common side

In [39]:
print(['\033[255;0;0;1m' + i + '\033[0m' for i in substrings])

['\x1bDate: November 20\n\n\n\x1b', '\x1bDate: Oct 20\n\x1b', '\x1bDate : Sept 20\n\x1b', '\x1bDate : August 20\n\n\x1b']


In [42]:
[i for i in substrings]
['\033[255;0;0;1mDate: November 20\n\n\n\033[0m', '\x1b[255;0;0;1mDate: Oct 20\n\x1b[0m', 
 '\x1b[255;0;0;1mDate : Sept 20\n\x1b[0m', '\x1b[255;0;0;1mDate : August 20\n\n\x1b[0m']

['\x1bDate: November 20\n\n\n\x1b',
 '\x1bDate: Oct 20\n\x1b',
 '\x1bDate : Sept 20\n\x1b',
 '\x1bDate : August 20\n\n\x1b']

In [6]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [8]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages_FULL(messages)
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Step 1:#### The user is asking a question about two specific products, the BlueWave Chromebook and the TechPro Desktop.\nStep 2:#### The prices of the two products are as follows:\n- BlueWave Chromebook: $249.99\n- TechPro Desktop: $999.99\nStep 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.\nStep 4:#### The assumption is incorrect. The TechPro Desktop is actually more expensive than the BlueWave Chromebook.\nResponse to user:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook costs $249.99 while the TechPro Desktop costs $999.99.",
        "role": "assistant"
      }
    }
  ],
  "created": 1687801504,
  "id": "chatcmpl-7VkYqlQDwAwtRESBYFr1C8lLYyB3r",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 153,
    "prompt_t

In [16]:
print(response.choices[0].message["content"])

Step 1:#### The user is asking a question about two specific products, the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### The prices of the two products are as follows:
- BlueWave Chromebook: $249.99
- TechPro Desktop: $999.99
Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is incorrect. The TechPro Desktop is actually more expensive than the BlueWave Chromebook.
Response to user:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook costs $249.99 while the TechPro Desktop costs $999.99.


'  \n\nDate: November 20\n\n\nFollowing the completion of induction chemotherapy, he is experiencing a mix of physical and emotional symptoms. Today his ANC 900, HCT 30\n\nPhysically, he is still recovering from the side effects of chemotherapy. Reports fatigue, weakness, and a general sense of low energy. Nausea and vomiting gradually subsiding. Has reduced appetite, and has lost 2 kg since initial diagnosis. Emotionally, the patient feeling some uncertainty as awaiting the results of the treatment response evaluation and bone marrow assessment. Will continue to provide supportive care, address any ongoing symptoms, and offer emotional support prn.\n\n  \n\nDate: Oct 20\nThis 65-year-old white male was \ndiagnosed with acute myeloid leukemia (AML) in August 2020. He underwent a round of induction chemotherapy in Aug 2020. Currently undergoing Consolidation therapy (in cycle 2 of 4). His ANC 1250, HCT 29\nHe is on Arabinosylcytosine and midostaurin \n\nCurrently, the patient is experie